In [ ]:
import os
import json
import random
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import transforms

# 1. Configuration & Directory Setup
OUTPUT_DIR = "Dataset-150\images"
IMAGES_DIR = os.path.join(OUTPUT_DIR, "Dataset Images")
LABELS_DIR = os.path.join(OUTPUT_DIR, "Labels")

os.makedirs(IMAGES_DIR, exist_ok=True)
os.makedirs(LABELS_DIR, exist_ok=True)

# 2. Randomization Constants (Cite: Assignment Requirements)
LINE_STYLES = ['-', '--', ':', '-.']
COLORS = ['red', 'blue', 'green', 'orange', 'purple', 'black', 'brown']
MARKERS = ['o', 's', '^', 'd', 'v']
FONT_SIZES = [10, 12, 14, 16]

def get_pixel_coords(ax, x_data, y_data):
    """Converts data coordinates to pixel coordinates relative to the image."""
    # Transform from data coords to display (pixel) coords
    display_coords = ax.transData.transform(np.vstack([x_data, y_data]).T)
    # Matplotlib origin is bottom-left, images (PIL/OpenCV) are top-left
    # We flip the Y coordinate based on the figure height
    fig_height = ax.figure.get_figwidth() * ax.figure.get_dpi()
    return display_coords

def generate_plot(image_id):
    fig, ax = plt.subplots(figsize=(10, 7), dpi=100)
    
    # Randomize number of lines (1-5) (Cite: Assignment Requirements)
    num_lines = random.randint(1, 5)
    all_line_data = []

    # Randomize global plot features
    ax.set_title(f"Experimental Results Sample {image_id}", fontsize=random.choice(FONT_SIZES))
    ax.set_xlabel("Time (h)" if random.random() > 0.5 else "Concentration", fontsize=12)
    ax.set_ylabel("Value", fontsize=12)

    for i in range(num_lines):
        # Generate random data points (Cite: Assignment Requirements)
        num_points = random.randint(5, 12)
        x = np.linspace(0, 100, num_points)
        y = np.sort(np.random.rand(num_points) * 50)  # Random ascending curve
        
        # Randomize style (Cite: Assignment Requirements)
        color = random.choice(COLORS)
        style = random.choice(LINE_STYLES)
        marker = random.choice(MARKERS)
        line_name = f"Group_{i+1}"

        # Error bar sizes (Cite: Assignment Requirements)
        top_err = np.random.rand(num_points) * 5
        bottom_err = np.random.rand(num_points) * 5
        
        # Plotting
        line_elements = ax.errorbar(x, y, yerr=[bottom_err, top_err], 
                                    fmt=style+marker, color=color, 
                                    capsize=5, label=line_name)

        # Extraction of Pixel Coordinates
        # 1. Point Centers
        pixels = ax.transData.transform(np.vstack([x, y]).T)
        # 2. Top Bar Ends
        top_pixels = ax.transData.transform(np.vstack([x, y + top_err]).T)
        # 3. Bottom Bar Ends
        bottom_pixels = ax.transData.transform(np.vstack([x, y - bottom_err]).T)

        points_list = []
        fig_h = fig.get_figheight() * fig.get_dpi()

        for j in range(num_points):
            # JSON format requires pixel distances (Cite: Sample JSONs)
            points_list.append({
                "x": float(pixels[j][0]),
                "y": float(fig_h - pixels[j][1]), # Flip Y for image space
                "topBarPixelDistance": float(abs(top_pixels[j][1] - pixels[j][1])),
                "bottomBarPixelDistance": float(abs(bottom_pixels[j][1] - pixels[j][1])),
                "label": ""
            })

        all_line_data.append({
            "label": {"lineName": line_name},
            "points": points_list
        })

    # Save Image
    img_filename = f"synthetic_{image_id}.png"
    plt.savefig(os.path.join(IMAGES_DIR, img_filename))
    plt.close(fig)

    # Save JSON (Cite: Assignment Requirements)
    json_filename = f"synthetic_{image_id}.json"
    with open(os.path.join(LABELS_DIR, json_filename), 'w') as f:
        json.dump(all_line_data, f, indent=2)

# 3. Execution Loop
TOTAL_IMAGES = 3000
print(f"Starting generation of {TOTAL_IMAGES} images...")
for i in range(TOTAL_IMAGES):
    if i % 100 == 0: print(f"Generated {i} images...")
    generate_plot(i)
print("Done!")